In [11]:
#%pip install git+https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-Datasets.git
#%pip install tqdm ipywidgets
#%pip install gymnasium
#%pip install torch

In [12]:
import logging as _logging_
_logging_.basicConfig(level='INFO')
#from energyplus.ooep.addons.logging import LogProvider

import energyplus.ooep as _ooep_

from energyplus.ooep.addons.display import ProgressProvider

from energyplus.ooep import (
    Simulator,
    Model,
    Weather,
    Report,
)

from energyplus.dataset.basic import dataset as _epds_

simulator = Simulator().add(
    ProgressProvider(),
    #LogProvider(),
)

await simulator.awaitable.run_forever(
    input=Simulator.InputSpecs(
        environment=Model().open(
            _epds_.models / 'ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
        ),
        weather=Weather().open(_epds_.weathers / 'USA_FL_Tampa.Intl.AP.722110_TMY3.epw'),
    ),
    options=Simulator.RuntimeOptions(
        #design_day=True,
    ),
)

  0%|          | 0/100 [00:00<?, ?it/s]

<Task pending name='Task-8' coro=<Engine.run_forever() running at /home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/awaitables.py:22>>

In [13]:
import numpy as _numpy_
import gymnasium as _gymnasium_

from energyplus.ooep.addons.rl.gymnasium.spaces import VariableBox
from energyplus.ooep.addons.rl.ray import SimulatorEnv

from energyplus.ooep.components.variables import (
    Actuator,
    OutputVariable,
)
from energyplus.ooep.addons.rl.gymnasium.core import BaseThinEnv

class TestThinEnv(BaseThinEnv):
    @property
    def action_space(self):
        return _gymnasium_.spaces.Dict({
            'thermostat': VariableBox(
                low=20., high=30.,
                dtype=_numpy_.float32,
                shape=(),
            ).bind(Actuator.Specs(
                type='Zone Temperature Control',
                control_type='Cooling Setpoint',
                key='CORE_MID',
            ))
        })
    
    @property
    def observation_space(self):
        return _gymnasium_.spaces.Dict({
            'temperature': VariableBox(
                low=-_numpy_.inf, high=+_numpy_.inf,
                dtype=_numpy_.float32,
                shape=(),
            ).bind(OutputVariable.Specs(
                type='People Air Temperature',
                key='CORE_MID',
            )),
        })
    
simulator.add(
    env := TestThinEnv()
)

# TODO mv testing
_ = '''
env.observe()
env.action_space.sample()['thermostat']
env.act(env.action_space.sample())
'''